# PROJEKAT

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import dlib

In [3]:
def loadImageAnd2Gray(path):
    img_haar = cv2.imread(path)
    return cv2.cvtColor(img_haar, cv2.COLOR_BGR2GRAY)

In [4]:
def display_gray_image(image):
    plt.imshow(image, 'gray')
    
def display_image(image):
    plt.imshow(image)

In [5]:
def draw_coords(image, coords):
    cv2.drawContours(image, [coords], -1, (255,255,255), 3)

In [6]:
def to_np_array(pose_landmarks, el_num=68, dtype="int"):
    coords = np.zeros((el_num, 2), dtype=dtype)
    
    for i in range(0, el_num):
        coords[i] = (pose_landmarks.part(i).x, pose_landmarks.part(i).y)
    
    return coords

In [45]:
def draw_circle(image, coords):
    cv2.circle(img, (coords[0], coords[1]), 25, (255, 255, 255), 2)

# Korisne stvari
cv2.rectangle(img, (x,y), (x+w, y+h), (0, 0 , 255), 3) -> iscrtavanje faca na ulaznoj slici

# Preoznavanje face sa haar-om

In [10]:
img = loadImageAnd2Gray('git_1.jpg')

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=4, minSize=(120, 120), flags=cv2.CASCADE_SCALE_IMAGE)
print("-Faces found: {}".format(len(faces)))

i = 0
for (x, y , w ,h) in faces:
    face_rect = dlib.rectangle(x, y, x + w, y + h)
    aligned_face = align_face(face_rect)
    if i == 10:
        display_gray_image(aligned_face)
    i += 1

-Faces found: 1


# Prepoznavanje face sa hog-om

In [11]:
img = loadImageAnd2Gray('git_1.jpg')

face_detector = dlib.get_frontal_face_detector()
faces = face_detector(img, 1)
print("-Faces found: {}".format(len(faces)))

i = 0
for face_rect in faces:
    aligned_face = align_face(face_rect)
    if i == 10:
        display_gray_image(aligned_face)
    i += 0

-Faces found: 2


# Pronalazenje 68 tacaka i cetriranje

In [8]:
predictor_model = "shape_predictor_68_face_landmarks.dat"
face_pose_predictor = dlib.shape_predictor(predictor_model)
DESIRED_LEFT_EYE = (0.35, 0.35)
DESIRED_RIGHT_EYE = (0.65, 0.35)
DESIRED_FACE_WIDTH = 256
DESIRED_FACE_HEIGHT = 256

In [9]:
def align_face(face_rect, verbose=False):

    pose_landmarks = face_pose_predictor(img, face_rect)
    landmark_coords = to_np_array(pose_landmarks)

    left_eye_coords = landmark_coords[36:41]
    right_eye_coords = landmark_coords[42:47]
    
    if verbose:
        draw_coords(img, landmark_coords)
        draw_coords(img, left_eye_coords)
        draw_coords(img, right_eye_coords)

    left_eye_center = left_eye_coords.mean(axis=0).astype("int")
    right_eye_center = right_eye_coords.mean(axis=0).astype("int")

    dx = right_eye_center[1] - left_eye_center[1]
    dy = right_eye_center[0] - left_eye_center[0]
    angle = np.degrees(np.arctan2(dx, dy)) # - 180

    dist = np.sqrt((dx * dx) + (dy * dy))
    desiredDist = (DESIRED_RIGHT_EYE[0] - DESIRED_LEFT_EYE[0])
    desiredDist *= DESIRED_FACE_WIDTH
    scale = desiredDist / dist

    eyes_center = ((left_eye_center[0] + right_eye_center[0]) // 2, (left_eye_center[1] + right_eye_center[1]) // 2)
    M = cv2.getRotationMatrix2D(eyes_center, angle, scale)
    tx = DESIRED_FACE_WIDTH * 0.5
    ty = DESIRED_FACE_HEIGHT * DESIRED_LEFT_EYE[1]
    M[0, 2] += (tx - eyes_center[0])
    M[1, 2] += (ty - eyes_center[1])

    (w, h) = (DESIRED_FACE_WIDTH, DESIRED_FACE_HEIGHT)
    aligned_face = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC)

    return aligned_face

# Ekstrakcija obelezja hog-om

In [ ]:
hog = cv2.HOGDescriptor()

In [100]:
img_hog = loadImageAnd2Gray('git_1.jpg')
face_images = []

for (x, y , w ,h) in faces:
    face_img = img[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (100, 110), interpolation=cv2.INTER_NEAREST)
    face_images.append(face_img)

# display_gray_image(face_images[2])

In [10]:
nbins = 9             # broj binova
cell_size = (8, 8)    # broj piksela po celiji
block_size = (3, 3)   # broj celija po bloku

img_for_shapes = face_images[0] 

hog = cv2.HOGDescriptor(_winSize=(img_for_shapes.shape[1] // cell_size[1] * cell_size[1], 
                                  img_for_shapes.shape[0] // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)